Скачиваем датасет с Я.диска

In [ ]:
# pragma dataset init Renopaty --size 4Gb

from io import BytesIO
from urllib.parse import urlencode
from zipfile import ZipFile

import requests

base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"
public_key = "https://disk.yandex.ru/d/uHaTRuAlbdbpdQ"

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()["href"]
response = requests.get(download_url)

dist_path = "/home/jupyter/mnt/datasets/Renopaty/"
zipfile = ZipFile(BytesIO(response.content))
zipfile.extractall(path=dist_path)

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from PIL import Image

In [ ]:
test_paths = os.path.join("/home/jupyter/datasphere/datasets/Renopaty/Test")
train_paths = os.path.join("/home/jupyter/datasphere/datasets/Renopaty/Train")

In [ ]:
for i in range(5):
    test_paths_c = f"/home/jupyter/datasphere/datasets/Renopaty/Test/{i}"
    train_paths_c= f"/home/jupyter/datasphere/datasets/Renopaty/Train/{i}"
    print(len(os.listdir(test_paths_c)), f"images for training in class {i}")
    print(len(os.listdir(train_paths_c)), f" images for testing in class {i}")

712 images for training in class 0
6408  images for testing in class 0
281 images for training in class 1
2532  images for testing in class 1
389 images for training in class 2
3502  images for testing in class 2
319 images for training in class 3
2879  images for testing in class 3
300 images for training in class 4
2709  images for testing in class 4


In [ ]:
batch_size = 32
image_size = (224, 224)

In [ ]:
def load_img(path):
    img = Image.open(path)
    img = img.resize(image_size)
    img_np = np.array(img)
    img_np = img_np/255
    return img_np

In [ ]:
test_x = []
test_y = []
for i in os.listdir(test_paths):
    for j in os.listdir(os.path.join(test_paths,i)):
        path = os.path.join(test_paths, str(i), j)
        img = load_img(path)
        test_y.append(int(i))
        test_x.append(img)
        # print(path)

In [ ]:
train_x = []
train_y = []
for i in os.listdir(train_paths):
    for j in os.listdir(os.path.join(train_paths, i)):
        path = os.path.join(train_paths, i, j)
        img = load_img(path)
        train_y.append(int(i))
        train_x.append(img)
        # print(path)

In [ ]:
train_x = np.array(train_x)
test_x = np.array(test_x)
train_y = np.array(train_y)
test_y = np.array(test_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(18030, 224, 224, 3)
(18030,)
(2001, 224, 224, 3)
(2001,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, SpatialDropout2D
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
model = Sequential()

#1
model.add(BatchNormalization(input_shape=(224,224 ,3)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.3))
#2
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.3))
#3
#5
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(152, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(5, activation='softmax'))

#Input = tf.keras.Input(shape=(224, 224, 3))
#x = preprocess_input(Input)
#x = model(Input,training=True)
#Resnet_model = Model(inputs=Input, outputs=[x], name='5Classes')

In [ ]:
def plot_graph(train_acc, val_acc, train_loss, val_loss):
  '''
  Функция выведет на экран графики точности и ошибки на проверочной и обучающей выборках.
  '''
  sns.set(style='darkgrid', palette='dark')
  plt.figure(figsize=(16, 10))
  plt.subplot(2, 2, 1)
  plt.title('Точность', fontweight='bold')
  plt.plot(train_acc, label='Точность на обучащей выборке')
  plt.plot(val_acc, label='Точность на проверочной выборке')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Доля верных ответов')
  plt.legend()

  plt.subplot(2, 2, 2)
  plt.title('Ошибка', fontweight='bold')
  plt.plot(train_loss, label='Ошибка на обучающей выборке')
  plt.plot(val_loss, label='Ошибка на проверочной выборке')
  plt.xlabel('Эпоха обучения')
  plt.ylabel('Ошибка')
  plt.legend()

  plt.show()

In [ ]:
import gc
gc.collect()

437

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger

In [ ]:
base_learning_rate = 0.0001
# Компилируем модель:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),  # Оптимизатор Adam c заданным ранее шагом обучения
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Ошибка SparseCategoricalCrossentropy
              metrics=['accuracy'])

In [ ]:
epochs = 50


#Задаем коллбеки
history_path = '/home/jupyter/datasphere/project/model_history_log2.csv'
weights_path = '/home/jupyter/datasphere/project/MobileNetV2.keras'

csv_logger = CSVLogger(history_path, append=True)
# уменьшение lr
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1)
# сохранение весов
model_checkpoint = ModelCheckpoint(weights_path, monitor='val_loss', save_best_only=True, verbose=1)


Retina_Resnet = model.fit(x=train_x,
                                 y=train_y,
                                 validation_data=(test_x,test_y),# Обучающая выборка
                                 epochs=epochs,
                                 shuffle=True,
                                 callbacks=[csv_logger,
                                   reduceLROnPlateau,
                                   model_checkpoint])   # Проверочная выборка

                                                    # После того, как пройдет обучение, выведем графики точности и ошибки:
plot_graph(Retina_Resnet.history['accuracy'],
           Retina_Resnet.history['val_accuracy'],
           Retina_Resnet.history['loss'],
           Retina_Resnet.history['val_loss'])

2024-04-13 17:03:04.026328: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 10856079360 exceeds 10% of free system memory.
2024-04-13 17:03:14.398112: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 10856079360 exceeds 10% of free system memory.


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2024-04-13 17:03:24.848254: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-13 17:03:25.461567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2024-04-13 17:03:27.4990

564/564 [==============================] - ETA: 0s - loss: 1.4028 - accuracy: 0.4201

/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(



Epoch 1: val_loss improved from inf to 2.17194, saving model to /home/jupyter/datasphere/project/MobileNetV2.keras
564/564 [==============================] - 56s 80ms/step - loss: 1.4028 - accuracy: 0.4201 - val_loss: 2.1719 - val_accuracy: 0.1804 - lr: 1.0000e-04
Epoch 2/50
563/564 [============================>.] - ETA: 0s - loss: 1.2355 - accuracy: 0.4836
Epoch 2: val_loss improved from 2.17194 to 1.47671, saving model to /home/jupyter/datasphere/project/MobileNetV2.keras
564/564 [==============================] - 43s 76ms/step - loss: 1.2354 - accuracy: 0.4836 - val_loss: 1.4767 - val_accuracy: 0.3823 - lr: 1.0000e-04
Epoch 3/50
563/564 [============================>.] - ETA: 0s - loss: 1.1936 - accuracy: 0.4993
Epoch 3: val_loss improved from 1.47671 to 1.29471, saving model to /home/jupyter/datasphere/project/MobileNetV2.keras
564/564 [==============================] - 43s 76ms/step - loss: 1.1936 - accuracy: 0.4993 - val_loss: 1.2947 - val_accuracy: 0.4563 - lr: 1.0000e-04
Epoc

KeyboardInterrupt: 